# Overview of Assignment 2

This assignment explores fundamental concepts and techniques for image retrieval. The main objective is to develop an image retrieval system that efficiently searches a database to find visually similar images to a query image. This will involve implementing and comparing different feature extraction methods, similarity measures, and indexing structures covered in lectures.

#   Enter your details below

# Name

# Banner ID

# Q1: Setting up the libraries and environment

# Q2: Feature Extraction

# Q3 Similarity Measures


# Q4 Image Retrieval Pipeline


In [ ]:
import numpy as np
import cv2
from sklearn.cluster import KMeans

def extract_sift_features(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Create a SIFT feature extractor
    sift = cv2.SIFT_create()
    
    # Detect and compute SIFT features
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    # print("keypoint :",keypoints)
    # print("descriptors :",descriptors)
    
    return descriptors

def cluster_features(features, k):
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features)
    
    # Get the cluster centers (visual words)
    visual_words = kmeans.cluster_centers_
    
    return visual_words

# Q5 Vector Database Integration